# Tutorial 25: Modules and Python Files

## PHYS 2600

## T25.0 - Module demo

We'll begin with a simple demonstration of editing and loading modules, and some common properties and issues you might run into.  In class, follow along with me on the screen!  If you're doing this on your own, you should open the module file `my_module.py` in a separate window to look at while you're using it in this notebook.

Run the cell below to save a copy of the `my_module.py` file to your current working directory (`/content` if you're using Google Colab).

In [ ]:
import os
import urllib.request

remote_url = "https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/tutorials/tut25/my_module.py"
local_path = os.path.abspath("my_module.py")

if not os.path.exists(local_path):
    print(f"copying file to {local_path}")
    urllib.request.urlretrieve(remote_url, local_path)
else:
    print(f"{local_path} already exists")



Now you can open `my_module.py` in Colab by clicking the folder icon in the left sidebar to open the file browser and then clicking on `my_module.py` to open it in a new tab. If you don't see the file, try clicking the refresh button (⟳) near the top of the file browser.

Notice that the interface for editing a `.py` file in Jupyter is very different!  There are no cells and a much more limited menu at the top.  In fact, there's no way to run a `.py` file from the Jupyter web interface - you'll only be able to run the code by importing it into a notebook as a module, as we'll do now.

In the cell below, __run the line `import my_module`__.

In [ ]:
import my_module

Notice that line of text that printed out when we imported.  When we import a Python module, __all of the code is evaluated__, except for a special block called the "main" block (more on that shortly.)  This can lead to security concerns, since arbitrary code is run on import; on your own machine, you should be wary of using Python modules that you don't trust.

The name `my_module` is a variable like any other, so we can print it out to see information about it!  In this case, it will tell us where the module was imported from:

In [ ]:
my_module

To see more information on a module, we can use the magic `?` command:

In [ ]:
?my_module

Notice that __modules can have docstrings, just like functions do__ - and if they do, we'll see them when we use `?`.  Most modules are sufficiently big and complicated that they have whole webpages to document them, instead of relying on a short docstring - but this can be useful for smaller modules.

Whatever variables and functions a module makes available to us should generally be documented somewhere.  But occasionally, it can be useful to have Python itself tell us what variables are in a module's namespace.  To do so, we use the `dir()` built-in command on the module object:

In [ ]:
dir(my_module)

All of the names surrounded by double-underscores `__` are "hidden" - they aren't intended to be used outside of the module at all.  (The `__` prefix actually has syntactic significance in Python, but only for dealing with object-oriented programming, which we won't get into.  If you don't know what object-oriented programming means, don't worry about this remark.)

After those names, we see the ordinary variables that have been imported with `my_module`, which we can access in the usual way:

In [ ]:
print(my_module.some_data)
print(my_module.other_data)
my_module.some_func()

We can change these variables by reassigning to them now:

In [ ]:
my_module.some_data += 99
print(my_module.some_data)

but of course, __this doesn't change the original module__ - next time we import, the original value of 44 will be back.  (It would be pretty odd if it did change the original, since it would mean somehow changing the lines of code in the `my_module.py` file!)

One of the statements in the module file is an import with an alias:

```python
import numpy as my_np
```

Remember, importing a module runs all the code in it, which includes importing other modules.  We can now use NumPy without importing it again, under the name `my_module.my_np`:

In [ ]:
print(my_module.my_np)

z = my_module.my_np.array([4, 5, 6])
print(my_module.my_np.sum(z * 3))

Of course, there's nothing stopping us from importing NumPy at the top level, and in fact it's probably less confusing (and certainly less typing) to do so.  If we do, it's _exactly the same module_ under different names, which means we don't need to worry about which "version" of NumPy we're using:

In [ ]:
import numpy as np

print(np.mean(z))
print(my_module.my_np.argmin(np.array([-3, 0, 3])))

At the end of the module file, there is some code that __wasn't__ run on import: everything in the block under `if __name__ == '__main__':`.  Those prints didn't run on import, and the variable `main_data` created in the block is not available here:

In [ ]:
my_module.main_data

This "main block" is intended to contain code to be run only when the module is run as a __script__ - a stand-alone Python program, separate from any notebook.  We don't have any way to run scripts with the Jupyter web interface, so we won't explore them further in this class.

What about __making changes__ to the module?  Let's try changing `y` to an array of letters instead:

```python
y = my_np.array(['i','j','k'])
```

Comment out the `y` assignment in `my_module.py` and replace it with this, then make sure to __save the module file__.  Run the cell below as-is and notice that `my_module.y` _hasn't_ changed in here:

In [ ]:
# import my_module

print(my_module.y)

We expect this behavior, because the local name `my_module.y` was pointed to the numeric array `[4,5,6]` when we imported the module, and we haven't imported again since we changed the code.

Now uncomment the import line, and notice that `my_module.y` _still_ hasn't updated!  Once again, this is an intentional safety feature of Python: __modules can only be imported once per program/kernel__, and subsequent `import` statements are simply ignored.

Finally, restart the kernel and run it one more time, to finally see `y` change to the array of letters we edited in.  (If it doesn't change after restarting the kernel, then you probably didn't save the changes to the `my_module.py` file - go back and try to save again.)

As I mentioned in lecture, there are also ways to reload a module without restarting the kernel.  In Python 3, we can import the `importlib` module to do it:

```python
import importlib
importlib.reload(my_module)
```

would work without restarting the kernel.  However, I think it's usually better practice to get a fresh kernel every time you edit the module, to avoid mistakes having to do with temporary variables persisting between edits.

Now __restart the kernel again__; we'll try importing the module using the `from` keyword, to see what is different.

In [ ]:
from my_module import *

We can now access all of the data and functions from `my_module` directly:

In [ ]:
print(other_data)
some_func()

We also get direct access to any imports that were done in `my_module` this way:

In [ ]:
my_np.sum(my_np.arange(10))

As we've discussed, the `from ... import *` "wildcard" import is usually discouraged; not having the module name attached can make your code harder to understand, and it's easy to accidentally overwrite local variables if you're not sure what everything in the module is named.

However, one exception that I like to use is for "driver" modules, that exist mostly to collect together import statements, e.g. I might create a file like `project_common.py` that would look like:

```python
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate.ode
...
```

and so on.  Then if I have multiple other Python files and notebooks in my project, I only need to `from project_common import *` at the top of them, instead of having lots of separate import blocks.

## T25.1 - Animated solutions to the Schrödinger equation

To get some experience with making and using modules, your task now is to __create a module called `solve_tise`__ which contains the implementation you wrote for tutorial 14 of a solver for the time-independent Schrodinger equation.  Specifically, it will need to contain the function `solve_infinite_SW()` and the functions and imports needed for _that_ function to work.  (In case you don't have your own completed code available, you can use the one in the solutions <a href="https://colab.research.google.com/github/wlough/CU-Phys2600-Fall2025/blob/main/tutorials/tut14/tut14_sol.ipynb" target="_blank" rel="noopener"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" /> </a>)

Once you've created the module file successfully and modified the code following the directions below, you'll be able to run the cells at the end to make _animated pictures_ of the shooting algorithm in action!

Create a file called `solve_tise.py` in your current working directory. In Colab you can do this by clicking the folder icon in the left sidebar to open the file browser, clicking the “New file” icon, naming it `solve_tise.py`, and then pasting your code into the new file and saving it. Once you've created the file, try running the import cell below. If you see `ModuleNotFoundError`, make sure `solve_tise.py` is saved in your current working directory (`/content` in Google Colab).



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import solve_tise

In [ ]:
def plot_psi_well():
    plt.axvline(0.0, color="k")
    plt.axvline(1.0, color="k")
    plt.axhline(0.0, color="k")

    plt.xlabel("x (nm)")
    plt.ylabel("$\\psi(x)$")


## How to make animated plots in Jupyter

Let's start by visualizing how the shooting solver actually converges to the answer!  This will be nicest to look at as an _animation_.  How do we make animated plots in pyplot?  There are [a couple of examples](https://matplotlib.org/gallery/animation/dynamic_image2.html) if we [search around the Internet](https://stackoverflow.com/questions/43445103/inline-animations-in-jupyter).  

The keys are the `animation` modules in `matplotlib` itself, and the `to_html5_video()` function that will let us draw an animation right in the Jupyter notebook itself. 



In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

# We need the "ax" object to make this work properly
fig, ax = plt.subplots()


def plot_sine_wave(phase):
    x = np.linspace(0, 2, 120)
    y = np.sin(2 * np.pi * (x - phase))
    frame = ax.plot(x, y, color="k")  # ax.plot returns an object for each frame

    return frame


frames = []
for phi in np.linspace(0, 2, 120):
    frames.append(plot_sine_wave(phi))

# Options: blit=True makes the animation smoother, interval=50 is a 50 ms/frame timing
ani = animation.ArtistAnimation(fig, frames, interval=25, blit=True, repeat=True)

plt.close(fig)  ## Stops Jupyter from showing the last frame alongside the animated plot
HTML(ani.to_html5_video())

## Animating the shooting solution to the TISE

Now back to our solver.  We'll need a modified version of `solve_infinite_SW` that will save a frame every shot, so we can animate at the end.

__Here are your instructions for patching `solve_infinite_SW` so that it will return animated frames:__

1. Include the line `import matplotlib.pyplot as plt` at the top of your `solve_tise.py` module.

2. Add the following setup code as the first two lines of `solve_infinite_SW()`:

```
fig, ax = plt.subplots()
frames = []
```

3. Add the following code to within the sub-function `delta_boundary(E)`, assuming that the variable `psi_E` contains the solution from the function `shoot_infinite_SW()` for the given value of `E`:

```
# Plot the shot!
frame = ax.plot(x, psi_E, label=E)
# Draw text on the plot to show E.  transform=ax.transAxes uses relative position
# instead of absolute x,y coordinates.
text = ax.text(0.1, 0.9, f'E={E:.2g}', transform=ax.transAxes)
frames.append(frame + [text])
```

4. Finally, change the return statement to read:

```
return E, psi_E, fig, frames
```

__Once you have made all of the changes above, restart the kernel and re-run from the top of this problem to reimport your module.__  Then run the cells below and watch your shooting code in action!

In [ ]:
x = np.linspace(0, 1, 1000)
Vx = np.zeros_like(x)
E, psi, fig, frames = solve_tise.solve_infinite_SW(0.5, x, Vx)

ani = animation.ArtistAnimation(fig, frames, interval=500, blit=True, repeat=False)
plot_psi_well()


plt.close(fig)  ## Stops Jupyter from showing the last frame alongside the animated plot
HTML(ani.to_html5_video())

# If your plots worked, you solved it!

And a more interesting shot with more steps:

In [ ]:
x = np.linspace(0, 1, 1000)
Vx = 0.6 * x
E, psi, fig, frames = solve_tise.solve_infinite_SW(12, x, Vx)

ani = animation.ArtistAnimation(fig, frames, interval=500, blit=True, repeat=False)
plot_psi_well()


plt.close(fig)  ## Stops Jupyter from showing the last frame alongside the animated plot
HTML(ani.to_html5_video())

Finally, here's an example showing some instability where exponential solutions start to appear with a big linear potential.  Notice how unstable the solution is!  Eventually, the solver settles onto the correct solution - a wave bunched up towards the right-hand side of the cell, where the potential $V(x) = 20x$ is large.  (Large potential means small kinetic energy, so the particle in a box spends most of its time to the right.)

In [ ]:
x = np.linspace(0, 1, 1000)
Vx = 20 * x
E, psi, fig, frames = solve_tise.solve_infinite_SW(11, x, Vx)

ani = animation.ArtistAnimation(fig, frames, interval=500, blit=True, repeat=False)
plot_psi_well()


plt.close(fig)  ## Stops Jupyter from showing the last frame alongside the animated plot
HTML(ani.to_html5_video())

## T25.2 - A quick intro to Python classes (optional)

For those of you who are curious about how classes and object-oriented programming work in Python, I thought I would include an optional exercise that goes through some of the basics.

Here we'll use the "OO" paradigm to do some examples related to working with two-dimensional points.  As a starting point, here is an array of points as a list of lists:

In [ ]:
pts = [
    [0.76135076, -0.77039654],
    [0.08163647, -0.10009183],
    [0.68868096, 0.91933837],
    [0.45523078, 0.38878166],
    [0.06226641, -0.71717507],
    [0.8791983, 0.56849504],
    [0.6159388, 0.2962535],
    [0.9074213, -0.07573378],
    [0.76437917, 0.00316327],
    [-0.28816238, -0.78560483],
    [0.2146455, -0.67439846],
    [0.17569189, 0.66686065],
    [-0.0185947, 0.6090448],
    [0.25339971, 0.86800028],
    [-0.45825946, 0.47931239],
    [0.33481938, 0.24627549],
    [-0.50618177, 0.00622503],
    [-0.83287995, 0.20805958],
    [-0.97785883, -0.84571404],
    [-0.968929, -0.27595801],
]

Instead of working with this directly as a list of lists (or a two-dimensional array), let's write a class to define an object representing a 2-d point.  (Then we can work with a list of points!)

### Part A

To define our own classes, we use the `class` keyword.  I've set up a basic class definition and some special functions in the cell below.

The `__init__` function is a special function called the __constructor__.  Whenever we create a new `vec2d()` object, this function is called.  Also note that whenever we write methods (functions that will be attached to objects and called with dot notation), the first argument is _always_ "`self`", which is a reference to the object which is calling the function.

Notice that in this case, the behavior of the constructor is fairly simple: it takes the two arguments `x` and `y` and saves them as properties of the created object.  We could have other code to set up our object in more complicated cases, but this is the bare minimum needed.

The second function should return `True` if the point is further from the origin than `r=1`.  __Complete the function.__ (Hint: you can access `x` and `y` coordinates inside the function definition using `self.x` and `self.y`.)

In [ ]:
class vec2d:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        # Special function to control how vec2d objects are printed out
        return f"({self.x:.3f}, {self.y:.3f})"

    def is_far(self):
        #

Now in the next cell, convert the list of lists `pts` to a __list of vec2d objects, `vec_pts`__.  

Notice that when you print `far_pts`, you see all of the points in the form `(x,y)` with 3 digits after the decimal points.  This is specified in the `__repr__` function above.  Try editing it and see how the results printed out below change!

In [ ]:
#

In [ ]:
print(vec_pts)

assert len(vec_pts) == 20
for pt in vec_pts:
    assert type(pt) is vec2d

Now __filter using your `is_far`__ function to get a list `far_pts` which are all greater than distance 1 from the origin.  

_(Hint: you can do this with a list comprehension, or just with a `for` loop and the accumulator pattern.  Because the function you want to use is a method, called with dot notation, it won't work properly with the `filter()` function.)_

In [ ]:
far_pts = [pt for pt in vec_pts if pt.is_far()]

In [ ]:
print(far_pts)
assert len(far_pts) == 5

### Part B

One of the more interesting things we can do when defining a new class is __operator overloading__.  We have a natural use of overloading in this example: we'd really like `+` to be overloaded to carry out the vector sum.  Of course, we have to do this explicitly: if you try to use `+` on two `vec2d` objects right now, you'll just get a syntax error.

To define the behavior of `+`, we have to define yet another special function.  In this case, we have to define the function `__add__` in our `vec2d` class.  This function should take two `vec2d` objects as arguments, and return a single object corresponding to the sum.

__Copy your class definition from part A__ into the cell below, and then __implement__ the `__add__` function.  Then run the following cell to verify that it gives you the expected result.

In [ ]:
#

In [ ]:
p1 = vec2d(1, 0)
p2 = vec2d(0, 1)
p1 + p2

Having `+` defined is convenient, since it extends to more complicated operations like using the `sum()` built-in function!  Now that you've overloaded `+`, compute the vector sum of all the points in `vec_pts` - it should be about (2.15, 1.01).

_(Note: by default, `sum()` begins with the numeric value 0 and then adds the results to it.  This will cause a TypeError here!  You can fix it in one of two ways:)_

1. Use the `start=` keyword argument of `sum()` to begin with the `vec2d` equivalent of zero;
2. Modify your `__add__` function so that it can deal with the number zero as a special case for one of the objects being passed.  For this case, the `isInstance(obj, class)` built-in function will probably be useful.

In [ ]:
#

### Part C

One last feature of object-oriented programming that Python includes is called __inheritance__.  Briefly, inheritance allows us to define a __sub-class__ from a starting class, which "inherits" all of the methods and properties of the parent class but then implements its own modifications.  The notation for this is to include the "parent" class or classes in parentheses when we use the `class` keyword.

For example, we can implement a `new_complex` class that will inherit the structure of `vec2d`, and then add some additional functions that only make sense for complex numbers.  I've done most of the work in the cell below for you: finish things up and __implement the `__mul__` function, which will overload the `*` operator__.  Then run the final cell to make sure it works!

In [ ]:
class new_complex(vec2d):

    def __init__(self, x, y):
        super().__init__(x, y)  # "Super-class" method calls the vec2d constructor

    def __repr__(self):
        if self.y < 0:
            pos_y = -self.y
            return f"{self.x:.2f} - {pos_y:.2f}i"
        else:
            return f"{self.x:.2f} + {self.y:.2f}i"

    def conjugate(self):
        return new_complex(self.x, -self.y)

    def __mul__(z1, z2):
        #

In [ ]:
z = new_complex(1, 1)
print(z)
print(z.conjugate())

print(new_complex(1, 1) * new_complex(1, -1))  # Should give 2
print(new_complex(1, 1) * new_complex(1, 1))  # Should give 2i